In [3]:
#importing packages needed
import requests
import pandas as pd
import json

In [5]:
df_customers = pd.read_csv('enriched_customer_file.csv')
df_customers.head()

,Unnamed: 0,name,age,job,incomeUSD,creditScore,ccNumber,married,hasChildren,height,...,address.countryCode,height_inches,weight_pounds,bmi,health,healthy_bmi_range,income level,region,state_population,per_capita_income
0,0,Dillon Muller,35,E-Sports Player,507663,789,3528-2138-9643-7767,False,False,156.0,...,US,61.417323,133.8435,24.94,Normal,18.5 - 25,Upper income,Midwest,5893718,"61,210"
1,1,Kimberlie Stracke,36,Astronomer,88150,421,6011-7789-3613-8614,False,False,156.0,...,US,61.417323,133.1820,24.82,Normal,18.5 - 25,Middle income,Northeast,13002700,"65,167"
2,2,Kenya Bergnaum,48,Market Research Analyst,73652,754,6011-6204-2025-0718-4734,False,True,156.0,...,US,61.417323,121.0545,22.56,Normal,18.5 - 25,Middle income,Midwest,11799448,"57,880"
3,3,Lauren Stoltenberg,23,E-Sports Player,506049,790,3680-065525-4234,False,False,151.0,...,US,59.448819,110.9115,22.06,Normal,18.5 - 25,Upper income,Midwest,11799448,"57,880"
4,4,Delena Kihn,55,Botanist,69762,634,3791-853663-90096,True,True,171.0,...,CO,67.322835,141.7815,21.99,Normal,18.5 - 25,Middle income,Northeast,7029917,"84,945"
